![alt text](example2.jpeg "Title")

In [1]:
import numpy as np

X = []
vals = [0,1]
for i in vals:
    for j in vals:
        for k in vals:
            X.append([i, j, k])
X.append([vals[0], vals[0], vals[0]])
X.append([vals[1], vals[1], vals[1]])
X.append([vals[1], vals[1], vals[1]])    
X = np.array(X)

Y3 = -6 * (1 - X[:, 2]) + 5 * (X[:, 2])
Y12 = 4 * (1 - X[:, 0]) * (1 - X[:, 1]) - 6 * X[:, 0] * (1 - X[:, 1]) \
    - 6 * (1 - X[:, 0]) * X[:, 1] + 3 * X[:, 0] * X[:, 1]
Y = Y3 + Y12

In [2]:
(Y3 * Y12).mean()

1.0

## Fit intercept.

In [3]:
f0 = np.mean(Y)
residual = Y - f0
f0

0.0

## Fit mains.

In [4]:
def fit_main(residual, idx):
    f = np.array([np.mean(residual[X[:, idx] == vals[i]]) for i in range(len(vals))])
    f_preds = [f[vals.index(x)] for x in X[:, idx]]
    residual = residual - f_preds
    return f, f_preds, residual

# Iterative algorithm, set the tolerance here.
# Smaller tolerance values result in smaller errors.
tol = 1e-10

updates = np.inf
residual = Y - f0
residual_prev = residual
f1, f1_preds, residual = fit_main(residual, 0)
f2, f2_preds, residual = fit_main(residual, 1)
f3, f3_preds, residual = fit_main(residual, 2)
while updates > tol:
    f1_new, f1_preds_new, residual = fit_main(residual, 0)
    f2_new, f2_preds_new, residual = fit_main(residual, 1)
    f3_new, f3_preds_new, residual = fit_main(residual, 2)
    updates = np.sum(np.abs(f1_preds_new)) + np.sum(np.abs(f2_preds_new)) + np.sum(np.abs(f3_preds_new))
    f1_preds += f1_preds_new
    f2_preds += f2_preds_new
    f3_preds += f3_preds_new
    f1 += f1_new
    f2 += f2_new
    f3 += f3_new

In [5]:
f1

array([ 0.04743083, -0.03952569])

In [6]:
f2

array([ 0.04743083, -0.03952569])

In [7]:
f3

array([-6.22529644,  5.18774704])

## Fit pairs.

In our context, we restrict the pairwise functions to be *pure*, so this backfitting scheme does not represent the scenario, as it learns any pairwise functions regardless of pureness.

In [8]:
# def fit_pair(residual, idx1, idx2):
#     f = np.array(
#         [[np.mean(residual[np.logical_and(X[:, idx1] == vals[i], X[:, idx2] == vals[j])]) for j in range(len(vals))] for i in range(len(vals))]
#     )
#     f_preds = [f[vals.index(xi), vals.index(xj)] for (xi, xj) in X[:, [idx1, idx2]]]
#     residual = residual - f_preds
#     return f, f_preds, residual

# updates = np.inf
# residual_prev = residual
# f12, f12_preds, residual = fit_pair(residual, 0, 1)
# f13, f13_preds, residual = fit_pair(residual, 0, 2)
# f23, f23_preds, residual = fit_pair(residual, 1, 2)
# while updates > tol:
#     f12_new, f12_preds_new, residual = fit_pair(residual, 0, 1)
#     f13_new, f13_preds_new, residual = fit_pair(residual, 0, 2)
#     f23_new, f23_preds_new, residual = fit_pair(residual, 1, 2)
#     updates = np.sum(np.abs(f12_preds_new)) + np.sum(np.abs(f13_preds_new)) + np.sum(np.abs(f23_preds_new))
#     f12_preds += f12_preds_new
#     f13_preds += f13_preds_new
#     f23_preds += f23_preds_new
#     f12 += f12_new
#     f13 += f13_new
#     f23 += f23_new

We use pure coding to implement sequential learning.
$$
\begin{pmatrix}4&-6\\-6&3\end{pmatrix}
$$
Forms a basis of 1-dimensional space of pure functions on $(x_1, x_2)$, and also $(x_1, x_3)$ and $(x_2, x_3)$ by symmetry. In order to learn $f_{12}, f_{13}, f_{23}\in\mathcal H_2$ (pure pairwise functions), they must be a linear mutiple of these basis vectors respectively.

In [9]:
b12 = 4 * (1 - X[:, 0]) * (1 - X[:, 1]) - 6 * X[:, 0] * (1 - X[:, 1]) \
    - 6 * (1 - X[:, 0]) * X[:, 1] + 3 * X[:, 0] * X[:, 1]
b13 = 4 * (1 - X[:, 0]) * (1 - X[:, 2]) - 6 * X[:, 0] * (1 - X[:, 2]) \
    - 6 * (1 - X[:, 0]) * X[:, 2] + 3 * X[:, 0] * X[:, 2]
b23 = 4 * (1 - X[:, 1]) * (1 - X[:, 2]) - 6 * X[:, 1] * (1 - X[:, 2]) \
    - 6 * (1 - X[:, 1]) * X[:, 2] + 3 * X[:, 1] * X[:, 2]

In [10]:
residual = residual_prev

In [11]:
residual

array([ -2.,   9., -12.,  -1., -12.,  -1.,  -3.,   8.,  -2.,   8.,   8.])

In [12]:
M = np.c_[b12, b13, b23]
c = np.linalg.inv(M.T @ M) @ M.T @ residual.reshape(-1, 1)

In [13]:
residual = residual - (M @ c).flatten()

In [14]:
f12 = c[0] * np.array([[4, -6], [-6, 3]])
f13 = c[1] * np.array([[4, -6], [-6, 3]])
f23 = c[2] * np.array([[4, -6], [-6, 3]])

## Evaluate the residual.

In [15]:
errors = residual
print(errors)

assert np.max(np.abs(errors)) < tol
print("Assertion Passed: All errors are smaller than tolerance, which was set to: {}".format(tol))

[-6.13664596  4.68594077 -5.70632442  5.28480491 -5.70632442  5.28480491
 -6.26215641  4.89751553 -6.13664596  4.89751553  4.89751553]


AssertionError: 

In [16]:
print("Learned functions:")
print("f0", f0)
print("f1", f1)
print("f2", f2)
print("f3", f3)
print("f12", f12)
print("f13", f13)
print("f23", f23)

Learned functions:
f0 0.0
f1 [ 0.04743083 -0.03952569]
f2 [ 0.04743083 -0.03952569]
f3 [-6.22529644  5.18774704]
f12 [[ 4.20761127 -6.3114169 ]
 [-6.3114169   3.15570845]]
f13 [[-0.03548265  0.05322398]
 [ 0.05322398 -0.02661199]]
f23 [[-0.03548265  0.05322398]
 [ 0.05322398 -0.02661199]]


In [17]:
print('\
True pure function:\n\
f0 0\n\
f1 [0, 0]\n\
f2 [0, 0]\n\
f3 [-6, 5]\n\
f12 [[4, -6],\n\
 [-6, 3]]\n\
f13 [[0, 0],\n\
 [0, 0]]\n\
f23 [[0, 0],\n\
 [0, 0]]\n\
')

True pure function:
f0 0
f1 [0, 0]
f2 [0, 0]
f3 [-6, 5]
f12 [[4, -6],
 [-6, 3]]
f13 [[0, 0],
 [0, 0]]
f23 [[0, 0],
 [0, 0]]



# Joint Learning
First, we show that learning the main with backfitting should learn the same results as using pure coding for main effects.

In [18]:
b1 = -6 * (1 - X[:, 0]) + 5 * X[:, 0]
b2 = -6 * (1 - X[:, 1]) + 5 * X[:, 1]
b3 = -6 * (1 - X[:, 2]) + 5 * X[:, 2]

In [19]:
Mm = np.c_[b1, b2, b3]
cm = np.linalg.inv(Mm.T @ Mm) @ Mm.T @ Y

In [20]:
f1_ = cm[0] * np.array([-6, 5])
f2_ = cm[1] * np.array([-6, 5])
f3_ = cm[2] * np.array([-6, 5])

In [21]:
print('f1={},f1_={}'.format(f1, f1_))
print('f2={},f2_={}'.format(f2, f2_))
print('f3={},f3_={}'.format(f3, f3_))

f1=[ 0.04743083 -0.03952569],f1_=[ 0.04743083 -0.03952569]
f2=[ 0.04743083 -0.03952569],f2_=[ 0.04743083 -0.03952569]
f3=[-6.22529644  5.18774704],f3_=[-6.22529644  5.18774704]


Now we do Joint learning with pure coding.

In [22]:
b12 = 4 * (1 - X[:, 0]) * (1 - X[:, 1]) - 6 * X[:, 0] * (1 - X[:, 1]) \
    - 6 * (1 - X[:, 0]) * X[:, 1] + 3 * X[:, 0] * X[:, 1]
b13 = 4 * (1 - X[:, 0]) * (1 - X[:, 2]) - 6 * X[:, 0] * (1 - X[:, 2]) \
    - 6 * (1 - X[:, 0]) * X[:, 2] + 3 * X[:, 0] * X[:, 2]
b23 = 4 * (1 - X[:, 1]) * (1 - X[:, 2]) - 6 * X[:, 1] * (1 - X[:, 2]) \
    - 6 * (1 - X[:, 1]) * X[:, 2] + 3 * X[:, 1] * X[:, 2]

In [23]:
Ma = np.c_[b1, b2, b3, b12, b13, b23]
ca = np.linalg.inv(Ma.T @ Ma) @ Ma.T @ Y

In [24]:
residuala = Y - Ma @ ca

In [25]:
errors = residuala
print(errors)

assert np.max(np.abs(errors)) < tol
print("Assertion Passed: All errors are smaller than tolerance, which was set to: {}".format(tol))

[ 0.00000000e+00  3.55271368e-15 -3.55271368e-15  6.66133815e-16
 -3.55271368e-15 -3.33066907e-16 -2.66453526e-15  8.88178420e-16
  0.00000000e+00  8.88178420e-16  8.88178420e-16]
Assertion Passed: All errors are smaller than tolerance, which was set to: 1e-10


In [26]:
f1_ = ca[0] * np.array([-6, 5])
f2_ = ca[1] * np.array([-6, 5])
f3_ = ca[2] * np.array([-6, 5])
f12_ = ca[3] * np.array([[4, -6], [-6, 3]])
f13_ = ca[4] * np.array([[4, -6], [-6, 3]])
f23_ = ca[5] * np.array([[4, -6], [-6, 3]])

In [27]:
print("Learned functions:")
print("f0", f0)
print("f1", f1_)
print("f2", f2_)
print("f3", f3_)
print("f12", f12_)
print("f13", f13_)
print("f23", f23_)

Learned functions:
f0 0.0
f1 [-9.99200722e-16  8.32667268e-16]
f2 [-4.99600361e-16  4.16333634e-16]
f3 [-6.  5.]
f12 [[ 4. -6.]
 [-6.  3.]]
f13 [[ 0. -0.]
 [-0.  0.]]
f23 [[ 0. -0.]
 [-0.  0.]]
